# **1 : Importing Libraries**

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from typing import Dict, List, Union, Optional
import logging
from datetime import datetime
import json
import yaml
import csv
import os
from scipy import stats

# **2 : EDA Module**

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class EDAModule:
    def __init__(self, df: pd.DataFrame, output_dir='eda_reports'):
        self.df = df
        os.makedirs(output_dir, exist_ok=True)
        self.output_dir = output_dir
        self.timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.numeric_cols = df.select_dtypes(include=[np.number]).columns
        self.categorical_cols = df.select_dtypes(include=['object']).columns
        self.insights = {}

    def generate_full_report(self) -> str:
        """Generate comprehensive EDA report with advanced insights."""
        self._check_scalability()
        self._advanced_statistical_analysis()
        self._analyze_distributions()
        self._detect_outliers()
        self._recommend_feature_engineering()
        self._analyze_correlations()
        return self._format_insights()

    def _check_scalability(self):
        """Perform scalability and performance checks."""
        self.insights['scalability'] = {
            'total_memory_usage': float(self.df.memory_usage(deep=True).sum() / 1024**2),
            'row_count': int(len(self.df)),
            'column_count': int(len(self.df.columns)),
            'sparsity_ratio': float(1 - (self.df.count().sum() / (self.df.shape[0] * self.df.shape[1])))
        }

    def _advanced_statistical_analysis(self):
        """Conduct advanced statistical analysis."""
        numeric_insights = {}
        for col in self.numeric_cols:
            numeric_insights[col] = {
                'skewness': float(self.df[col].skew()),
                'kurtosis': float(self.df[col].kurtosis()),
                'coefficient_of_variation': float(self.df[col].std() / self.df[col].mean()) if self.df[col].mean() != 0 else float('inf'),
                'quantiles': {
                    '25%': float(self.df[col].quantile(0.25)),
                    '50%': float(self.df[col].quantile(0.5)),
                    '75%': float(self.df[col].quantile(0.75))
                }
            }

        categorical_insights = {}
        for col in self.categorical_cols:
            categorical_insights[col] = {
                'unique_values_count': int(self.df[col].nunique()),
                'top_5_values': dict(self.df[col].value_counts().head()),
                'entropy': float(self._calculate_entropy(self.df[col]))
            }

        self.insights['statistical_analysis'] = {
            'numeric_columns': numeric_insights,
            'categorical_columns': categorical_insights
        }

    def _analyze_distributions(self):
        """Analyze distribution characteristics."""
        distribution_insights = {}
        for col in self.numeric_cols:
            _, p_value = stats.normaltest(self.df[col].dropna())
            distribution_insights[col] = {
                'is_normal_distribution': bool(p_value > 0.05),
                'shapiro_wilk_test_p_value': float(p_value)
            }
        self.insights['distribution_analysis'] = distribution_insights

    def _detect_outliers(self):
        """Detect outliers using IQR method."""
        outlier_insights = {}
        for col in self.numeric_cols:
            Q1 = self.df[col].quantile(0.25)
            Q3 = self.df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            outliers = self.df[(self.df[col] < lower_bound) | (self.df[col] > upper_bound)]
            outlier_insights[col] = {
                'outlier_count': int(len(outliers)),
                'outlier_percentage': float(len(outliers) / len(self.df) * 100),
                'lower_bound': float(lower_bound),
                'upper_bound': float(upper_bound)
            }

        self.insights['outlier_analysis'] = outlier_insights

    def _recommend_feature_engineering(self):
        """Provide feature engineering recommendations."""
        recommendations = []
        for col in self.numeric_cols:
            if self.insights['statistical_analysis']['numeric_columns'][col]['coefficient_of_variation'] > 1:
                recommendations.append(f"Consider binning column {col} due to high variability")
        for col in self.categorical_cols:
            if self.insights['statistical_analysis']['categorical_columns'][col]['unique_values_count'] > 10:
                recommendations.append(f"Consider advanced encoding for {col} (e.g., target encoding)")
        self.insights['feature_engineering_recommendations'] = recommendations

    def _analyze_correlations(self):
        """Analyze correlations between numerical columns."""
        if len(self.numeric_cols) > 1:
            correlation_matrix = self.df[self.numeric_cols].corr()
            high_correlations = []
            for i in range(len(self.numeric_cols)):
                for j in range(i + 1, len(self.numeric_cols)):
                    col1 = self.numeric_cols[i]
                    col2 = self.numeric_cols[j]
                    corr = correlation_matrix.loc[col1, col2]
                    if abs(corr) > 0.5:
                        high_correlations.append({
                            'columns': (col1, col2),
                            'correlation': float(corr)
                        })
            self.insights['correlation_analysis'] = high_correlations
        else:
            self.insights['correlation_analysis'] = "Insufficient numerical columns for correlation analysis"

    def _calculate_entropy(self, series):
        """Calculate entropy for categorical variable."""
        value_counts = series.value_counts(normalize=True)
        return -np.sum(value_counts * np.log2(value_counts))

    def save_insights(self, format='all'):
        """Save insights in multiple file formats."""
        base_filename = f"{self.output_dir}/eda_insights_{self.timestamp}"
        if format in ['json', 'all']:
            with open(f"{base_filename}.json", 'w') as f:
                json.dump(self.insights, f, indent=4)
        if format in ['yaml', 'yml', 'all']:
            with open(f"{base_filename}.yaml", 'w') as f:
                yaml.dump(self.insights, f, default_flow_style=False)
        if format in ['csv', 'all']:
            self._save_insights_to_csv(base_filename)
        if format in ['txt', 'text', 'all']:
            with open(f"{base_filename}.txt", 'w') as f:
                f.write(self._format_insights())
        return f"Insights saved in {base_filename}.*"

    def _save_insights_to_csv(self, base_filename):
        """Convert nested insights to flat CSV structure."""
        rows = []
        def flatten_dict(d, parent_key='', sep='_'):
            items = []
            for k, v in d.items():
                new_key = f"{parent_key}{sep}{k}" if parent_key else k
                if isinstance(v, dict):
                    items.extend(flatten_dict(v, new_key, sep=sep).items())
                else:
                    items.append((new_key, v))
            return dict(items)
        for section, content in self.insights.items():
            if isinstance(content, dict):
                flattened = flatten_dict({section: content})
                rows.append(flattened)
            elif isinstance(content, list):
                for item in content:
                    rows.append(flatten_dict({section: item}))
        if rows:
            keys = set().union(*[d.keys() for d in rows])
            with open(f"{base_filename}.csv", 'w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=sorted(keys))
                writer.writeheader()
                writer.writerows(rows)

    def _format_insights(self) -> str:
        """Format insights into a readable report."""
        report_sections = []
        scalability = self.insights['scalability']
        report_sections.append(f"""Scalability Analysis:
- Total Memory Usage: {scalability['total_memory_usage']:.2f} MB
- Total Rows: {scalability['row_count']}
- Total Columns: {scalability['column_count']}
- Data Sparsity Ratio: {scalability['sparsity_ratio']:.2%}""")
        report_sections.append("\nDetailed Statistical Analysis:")
        for col_type, columns in self.insights['statistical_analysis'].items():
            report_sections.append(f"\n{col_type.replace('_', ' ').title()}:")
            for col, stats in columns.items():
                report_sections.append(f"- {col}: {stats}")
        report_sections.append("\nDistribution Analysis:")
        for col, analysis in self.insights['distribution_analysis'].items():
            report_sections.append(f"- {col}: {'Normal' if analysis['is_normal_distribution'] else 'Non-Normal'} Distribution")
        report_sections.append("\nOutlier Analysis:")
        for col, outliers in self.insights['outlier_analysis'].items():
            report_sections.append(f"- {col}: {outliers['outlier_percentage']:.2f}% Outliers")
        report_sections.append("\nFeature Engineering Recommendations:")
        report_sections.append('\n'.join(self.insights['feature_engineering_recommendations']))
        return '\n'.join(report_sections)

    def run_and_save(self, formats=['json', 'yaml', 'txt']):
        """Run the full report generation and save insights in specified formats."""
        logger.info("Generating full EDA report...")
        self.generate_full_report()
        logger.info("Saving insights in formats: %s", formats)
        for fmt in formats:
            self.save_insights(format=fmt)
        logger.info("EDA report generation and saving completed.")








# **3 : Mathematical Query Module**

In [ ]:
class MathModule:
    def __init__(self, df: pd.DataFrame):
        self.df = df

    def execute_math_query(self, query: str) -> Union[float, str]:
        """Execute mathematical queries with error handling and validation."""
        try:
            if "difference between" in query.lower():
                col_name = self._extract_column_name(query)
                return self._calculate_difference(col_name)
            elif "standard deviation" in query.lower():
                col_name = self._extract_column_name(query)
                return self._calculate_std(col_name)
            elif "sum of values" in query.lower():
                return self._calculate_conditional_sum(query)

            return "Query not recognized. Please rephrase."

        except Exception as e:
            logger.error(f"Error in math query execution: {str(e)}")
            return f"Error processing query: {str(e)}"

    def _extract_column_name(self, query: str) -> str:
        """Extract column name from query string."""
        for col in self.df.columns:
            if col.lower() in query.lower():
                return col
        raise ValueError("Column name not found in query")

    def _calculate_difference(self, col_name: str) -> float:
        values = self.df[col_name].dropna().sort_values(ascending=False)
        if len(values) < 2:
            raise ValueError("Insufficient data points")
        return float(values.iloc[0] - values.iloc[1])

    def _calculate_std(self, col_name: str) -> float:
        return float(self.df[col_name].std())

    def _calculate_conditional_sum(self, query: str) -> float:
        threshold = float(query.split('greater than')[1].split()[0])
        col_name = self._extract_column_name(query)
        return float(self.df[self.df[col_name] > threshold][col_name].sum())



# **4 : DataQuery Module**

In [ ]:
class DataQueryModule:
    def __init__(self, df: pd.DataFrame):
        self.df = df

    def execute_query(self, query: str) -> pd.DataFrame:
        """Execute data queries with enhanced functionality."""
        try:
            if "top" in query.lower():
                return self._handle_top_query(query)
            elif "maximum" in query.lower() or "minimum" in query.lower():
                return self._handle_extreme_value_query(query)
            elif "group by" in query.lower():
                return self._handle_groupby_query(query)

            return pd.DataFrame({"error": ["Query not recognized"]})

        except Exception as e:
            logger.error(f"Error in data query execution: {str(e)}")
            return pd.DataFrame({"error": [str(e)]})

    def _handle_top_query(self, query: str) -> pd.DataFrame:
        parts = query.lower().split()
        n = int(parts[parts.index("top") + 1])
        col_name = self._extract_column_name(query)
        condition_value = float(query.split(">")[1].strip().split()[0])
        return self.df[self.df[col_name] > condition_value].head(n)

    def _handle_extreme_value_query(self, query: str) -> pd.DataFrame:
        col_name = self._extract_column_name(query)
        is_max = "maximum" in query.lower()
        return self.df[self.df[col_name] == (self.df[col_name].max() if is_max else self.df[col_name].min())]

    def _handle_groupby_query(self, query: str) -> pd.DataFrame:
        group_col = query.split("group by")[1].strip().split()[0]
        agg_col = self._extract_column_name(query)
        return self.df.groupby(group_col)[agg_col].agg(['mean', 'count', 'sum']).reset_index()

    def _extract_column_name(self, query: str) -> str:
        """Extract column name from query string."""
        for col in self.df.columns:
            if col.lower() in query.lower():
                return col
        raise ValueError("Column name not found in query")



# **5 : LLM Data Querying**

In [ ]:
class OpenQAModule:
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.model_name = "facebook/opt-350m"
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForCausalLM.from_pretrained(self.model_name)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)

    def get_direct_column_value(self, query: str) -> str:
        """Handle direct column value queries"""
        for col in self.df.columns:
            if col.lower() in query.lower():
                if 'highest' in query.lower() or 'maximum' in query.lower():
                    return str(self.df[col].max())
                elif 'lowest' in query.lower() or 'minimum' in query.lower():
                    return str(self.df[col].min())
                elif 'average' in query.lower():
                    return str(self.df[col].mean())
        return "Column not found or query not understood"

    def get_answer(self, question: str, context: str) -> str:
        try:
            prompt = f"""Context: {context}

Question: {question}

Answer:"""

            inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(self.device)
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=100,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True
            )
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return response.split("Answer:")[1].strip()
        except Exception as e:
            logger.error(f"Error in QA generation: {str(e)}")
            return f"Error processing query: {str(e)}"

class DataDrivenChatbot:
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.eda_module = EDAModule(df)
        self.eda_report = self.eda_module.generate_full_report()
        self.math_module = MathModule(df)
        self.data_query_module = DataQueryModule(df)
        self.qa_module = OpenQAModule(df)
        self.query_history = []

    def process_query(self, query: str) -> Union[str, pd.DataFrame]:
        try:
            self.query_history.append((datetime.now(), query))

            # Direct column value queries
            direct_value = self.qa_module.get_direct_column_value(query)
            if direct_value != "Column not found or query not understood":
                return direct_value

            if any(word in query.lower() for word in ['calculate', 'sum', 'average', 'deviation']):
                return self.math_module.execute_math_query(query)
            elif any(word in query.lower() for word in ['top', 'maximum', 'minimum', 'group by']):
                return self.data_query_module.execute_query(query)
            else:
                return self.qa_module.get_answer(query, self.eda_report)

        except Exception as e:
            logger.error(f"Error processing query: {str(e)}")
            return str(e)


if __name__ == "__main__":
    df = pd.read_csv('/content/Salary_Data.csv')
    eda = EDAModule(df)

    # Generate Full Report
    report = eda.generate_full_report()

    # Save Insights
    eda.save_insights()

    chatbot = DataDrivenChatbot(df)

    while True:
        user_query = input("Ask a question: ")
        if user_query.lower() == "exit":
            break
        response = chatbot.process_query(user_query)
        print(response)

Ask a question: Lowest Salary
37731
Ask a question: Average Age
27.216666666666665
Ask a question: Highest age
38.0
Ask a question: Show me the top 5 rows where column Age > 25
    YearsExperience   Age  Salary
16              5.1  26.0   66029
17              5.3  27.0   83088
18              5.9  28.0   81363
19              6.0  29.0   93940
20              6.8  30.0   91738
Ask a question: What is the maximum value in column Salary?
122391
Ask a question: What is the standard deviation of column Age?
5.161267108077196
Ask a question: exit
